# Week4授業課題
## 信用情報の学習
Kaggleの Home Credit Default Risk コンペティションの学習および推定を行います。

# 【問題1】
## コンペティション内容の確認
コンペティションのOverviewページ読み、「Home Credit Default Risk」について以下の観点について確認してください。

- 何を学習し、何を予測するのか
- どのようなファイルを作りKaggleに提出するか
- 提出されたものはどういった指標値で評価されるのか

## 何を学習し、何を予測するのか

## どのようなファイルを作りKaggleに提出するか

## 提出されたものはどういった指標値で評価されるのか

まずは単純な方法による **ベースラインモデル** を作成します。精度の基準となるモデルです。

精度を高くする必要はありません。エラーなく実行でき、Kaggle側に推定値を提出できることを目指します。

# 【問題2】
## 学習と検証
データを簡単に分析、前処理し、学習、検証するまでの一連の流れを作成・実行してください。

検証にはこのコンペティションで使用される評価指標を用いるようにしてください。学習に用いる手法は指定しません。

# 【問題3】
## テストデータに対する推定
テストデータ（`application_test.csv`）に対して推定を行い、Kaggleに提出を行ってください。

正しく提出が行えていれば、精度は低くても構いません。

In [ ]:
ベースラインモデルを元に、入力する特徴量に様々な工夫を行い精度を向上させていきます。

# 【問題4】
## 特徴量エンジニアリング
精度を上げるために以下のような観点で 特徴量エンジニアリング（Feature Engineering） を行ってください。

- どの特徴量を使うか
- どう前処理をするか

何をした時に検証データに対する評価指標がどのようになったかをまとめてください。最低5パターンの学習・検証を行ってください。

精度が高かったものに関してはテストデータに対しても推定を行い、Kaggleに提出を行ってください。